In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder

### Preprocessing

In [ ]:
# Load your dataset
df = pd.read_csv("final_flat_dataset.csv")  # replace with actual path
df.head()

,subject_id,ethnicity,insurance,religion,gender,dob,diagnosis_summary,procedure_summary,prescriptions
0,10006,BLACK/AFRICAN AMERICAN,Medicare,CATHOLIC,F,2094-03-05,Anemia of other chronic disease; Anticoagulant...,Hemodialysis; Injection or infusion of oxazoli...,"Acetaminophen, Alteplase (Catheter Clearance),..."
1,10011,UNKNOWN/NOT SPECIFIED,Private,CATHOLIC,F,2090-06-05,Acute and subacute necrosis of liver; Chronic ...,Parenteral infusion of concentrated nutritiona...,NaN
2,10013,UNKNOWN/NOT SPECIFIED,Medicare,CATHOLIC,F,2038-09-03,Atrial fibrillation; Cardiogenic shock; Mitral...,Arterial catheterization,"Acetaminophen, Alteplase (Catheter Clearance),..."
3,10017,WHITE,Medicare,CATHOLIC,F,2075-09-21,Acute posthemorrhagic anemia; Closed fracture ...,Partial shoulder replacement; Transfusion of p...,"1/2 NS, Acetaminophen, Albuterol, Albuterol 0...."
4,10019,WHITE,Medicare,CATHOLIC,M,2114-06-20,Acute alcoholic hepatitis; Acute kidney failur...,Continuous invasive mechanical ventilation for...,"1/2 NS, Albumin 25% (12.5gm), Artificial Tear ..."


In [ ]:
# Optional: Fill text fields
df['diagnosis_summary'] = df['diagnosis_summary'].fillna("")
df['ethnicity'] = df['ethnicity'].fillna("")
df['insurance'] = df['insurance'].fillna("")
df['religion'] = df['religion'].fillna("")
df['gender'] = df['gender'].fillna("")

84

In [11]:
# Transform features
X = preprocessor.fit_transform(df)

# Encode target
le = LabelEncoder()
y_encoded = le.fit_transform(df['procedure_summary'])

# One-hot encode target for MLP
y_onehot = np.eye(len(np.unique(y_encoded)))[y_encoded]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)

X_train.shape, y_train.shape

((70, 125), (70, 83))

### Model

In [ ]:
# -----------------------------
# Manual MLP Components
# -----------------------------

# Activation Functions
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=1, keepdims=True)

def cross_entropy_loss(y_pred, y_true):
    # y_true must be one-hot encoded
    m = y_true.shape[0]
    log_likelihood = -np.log(y_pred[range(m), y_true.argmax(axis=1)] + 1e-9)
    loss = np.sum(log_likelihood) / m
    return loss


In [ ]:

# MLP Initialization
def init_mlp(input_dim, hidden_dim, output_dim):
    params = {
        "W1": np.random.randn(input_dim, hidden_dim) * 0.01,
        "b1": np.zeros((1, hidden_dim)),
        "W2": np.random.randn(hidden_dim, output_dim) * 0.01,
        "b2": np.zeros((1, output_dim))
    }
    return params

# Forward Pass
def forward_pass(X, params):
    Z1 = np.dot(X, params["W1"]) + params["b1"]
    A1 = relu(Z1)
    Z2 = np.dot(A1, params["W2"]) + params["b2"]
    A2 = softmax(Z2)
    cache = {"Z1": Z1, "A1": A1, "Z2": Z2, "A2": A2}
    return A2, cache

# Backward Pass (Backprop)
def backward_pass(X, y_true, params, cache):
    m = X.shape[0]
    dZ2 = cache["A2"] - y_true
    dW2 = np.dot(cache["A1"].T, dZ2) / m
    db2 = np.sum(dZ2, axis=0, keepdims=True) / m

    dA1 = np.dot(dZ2, params["W2"].T)
    dZ1 = dA1 * relu_derivative(cache["Z1"])
    dW1 = np.dot(X.T, dZ1) / m
    db1 = np.sum(dZ1, axis=0, keepdims=True) / m

    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}
    return grads

# Update Parameters
def update_params(params, grads, lr):
    params["W1"] -= lr * grads["dW1"]
    params["b1"] -= lr * grads["db1"]
    params["W2"] -= lr * grads["dW2"]
    params["b2"] -= lr * grads["db2"]
    return params

# One Training Step 
def train_step(X, y_true, params, lr=0.01):
    y_pred, cache = forward_pass(X, params)
    loss = cross_entropy_loss(y_pred, y_true)
    grads = backward_pass(X, y_true, params, cache)
    params = update_params(params, grads, lr)
    return loss, params

### Training

In [ ]:
X_train.shape  
y_train.shape   #

(70, 83)

In [16]:
input_dim = X_train.shape[1]
hidden_dim = 32               # You can tune this
output_dim = y_train.shape[1]

params = init_mlp(input_dim, hidden_dim, output_dim)

In [ ]:
# X.shape = (num_samples, num_features)
# y.shape = (num_samples, num_classes), one-hot

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

params = init_mlp(input_dim=X.shape[1], hidden_dim=64, output_dim=y.shape[1])

for epoch in range(100):
    loss, params = train_step(X_train, y_train, params, lr=0.1)
    
    # Optional: Track training accuracy
    y_pred_probs, _ = forward_pass(X_train, params)
    accuracy = np.mean(np.argmax(y_pred_probs, axis=1) == np.argmax(y_train, axis=1))
    
    print(f"Epoch {epoch+1}, Loss: {loss:.4f}, Accuracy: {accuracy:.4f}")

Epoch 1, Loss: 4.4118
Epoch 2, Loss: 4.4100
Epoch 3, Loss: 4.4083
Epoch 4, Loss: 4.4067
Epoch 5, Loss: 4.4050
Epoch 6, Loss: 4.4033
Epoch 7, Loss: 4.4017
Epoch 8, Loss: 4.4001
Epoch 9, Loss: 4.3985
Epoch 10, Loss: 4.3969
Epoch 11, Loss: 4.3953
Epoch 12, Loss: 4.3937
Epoch 13, Loss: 4.3921
Epoch 14, Loss: 4.3906
Epoch 15, Loss: 4.3890
Epoch 16, Loss: 4.3875
Epoch 17, Loss: 4.3859
Epoch 18, Loss: 4.3844
Epoch 19, Loss: 4.3829
Epoch 20, Loss: 4.3813
Epoch 21, Loss: 4.3798
Epoch 22, Loss: 4.3783
Epoch 23, Loss: 4.3767
Epoch 24, Loss: 4.3752
Epoch 25, Loss: 4.3736
Epoch 26, Loss: 4.3721
Epoch 27, Loss: 4.3705
Epoch 28, Loss: 4.3690
Epoch 29, Loss: 4.3674
Epoch 30, Loss: 4.3658
Epoch 31, Loss: 4.3642
Epoch 32, Loss: 4.3625
Epoch 33, Loss: 4.3609
Epoch 34, Loss: 4.3592
Epoch 35, Loss: 4.3575
Epoch 36, Loss: 4.3557
Epoch 37, Loss: 4.3540
Epoch 38, Loss: 4.3522
Epoch 39, Loss: 4.3503
Epoch 40, Loss: 4.3484
Epoch 41, Loss: 4.3465
Epoch 42, Loss: 4.3445
Epoch 43, Loss: 4.3424
Epoch 44, Loss: 4.34